### Import Libraries

In [3]:
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler



### Load the Dataset

In [4]:
flight = pd.read_csv('Airline_Delay_Cause.csv')


### Drop Columns or Rows with evaluation sample output

In [5]:
test=flight.drop(['arr_delay'], axis=1)
test.sample(n=10, random_state=42).to_csv("test_sample.csv", index=False)
evaluation=flight
evaluation.sample(n=10, random_state=42).to_csv("eval_sample.csv", index=False)

### Drop Columns or Rows

In [6]:
# Drop missing values
flight.dropna(inplace=True)

# Convert categories
cat_cols = ['carrier', 'carrier_name', 'airport', 'airport_name']
for col in cat_cols:
    flight[col] = flight[col].astype('category')

# Convert counts to int
flight['arr_flights'] = flight['arr_flights'].astype(int)
flight['arr_del15'] = flight['arr_del15'].astype(int)


### Feature engineering

In [7]:
def feature_engineering_regressor(df, fillna=True):
    df = df.copy()

    df['arr_flights'] = df['arr_flights'].replace(0, np.nan)

    # Normalize rates
    df['delay_ratio'] = df['arr_del15'] / df['arr_flights']
    df['cancellation_rate'] = df['arr_cancelled'] / df['arr_flights']
    df['diversion_rate'] = df['arr_diverted'] / df['arr_flights']

    # Total delay
    delay_cols = ['carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay']
    df['total_delay'] = df[delay_cols].sum(axis=1)
    df['total_delay'] = df['total_delay'].replace(0, np.nan)

    # Delay % features
    for col in delay_cols:
        new_col = col.replace('_delay', '_delay_pct')
        df[new_col] = df[col] / df['total_delay']
    pct_cols = [col.replace('_delay', '_delay_pct') for col in delay_cols]
    if fillna:
        df[pct_cols] = df[pct_cols].fillna(0)

    # Year-Month
    df['year_month'] = df['year'].astype(str) + '-' + df['month'].astype(str).str.zfill(2)

    # Season
    df['season'] = df['month'].apply(lambda m: (
        'Winter' if m in [12, 1, 2] else
        'Spring' if m in [3, 4, 5] else
        'Summer' if m in [6, 7, 8] else
        'Fall'
    ))

    # Carrier total flights
    df['carrier_total_flights'] = df.groupby('carrier', observed=False)['arr_flights'].transform('sum')

    # Airport delay rate
    airport_delay_ratio = df.groupby('airport', observed=False).apply(
        lambda x: x['arr_del15'].sum() / x['arr_flights'].sum()
    )
    df['airport_delay_rate'] = df['airport'].map(airport_delay_ratio)

    # Disruption flag
    df['disrupted'] = ((df['arr_cancelled'] > 0) | (df['arr_diverted'] > 0)).astype(int)

    # Risk classification
    def classify_risk(ratio):
        if pd.isna(ratio):
            return np.nan
        elif ratio <= 0.20:
            return 0
        elif ratio <= 0.40:
            return 1
        else:
            return 2
    df['delay_risk_level'] = df['delay_ratio'].apply(classify_risk)

    # Mean delay per flight
    df['mean_delay_per_flight'] = df['total_delay'] / df['arr_flights']
    if fillna:
        df['mean_delay_per_flight'] = df['mean_delay_per_flight'].fillna(0)

    # Dominant delay cause
    df['dominant_delay_cause'] = df[delay_cols].idxmax(axis=1)

    # Monthly delay rate
    month_delay = df.groupby('month', observed=False).apply(
        lambda x: x['arr_del15'].sum() / x['arr_flights'].sum()
    )
    df['month_delay_rate'] = df['month'].map(month_delay)

    # Delay pressure
    df['carrier_vs_airport_ratio'] = df['carrier_delay_pct'] / (
        df['weather_delay_pct'] + df['nas_delay_pct'] + 1e-6
    )

    # Seasonal-airport combo
    df['season_airport_combo'] = df['season'].astype(str) + '_' + df['airport'].astype(str)
    season_airport_delay = df.groupby('season_airport_combo', observed=False).apply(
        lambda x: x['arr_del15'].sum() / x['arr_flights'].sum()
    )
    df['season_airport_delay_rate'] = df['season_airport_combo'].map(season_airport_delay)

    # ✅ Final NaN fill ONLY for numeric columns
    if fillna:
        numeric_cols = df.select_dtypes(include=[np.number]).columns
        df[numeric_cols] = df[numeric_cols].fillna(0)

    return df


### train,validation and test split then apply feature engineering

In [ ]:
from sklearn.model_selection import train_test_split

# ==============================================================

# --------------------------------------------------------------
# - Split BEFORE feature engineering
# - Target: arr_delay
# ==============================================================

# STEP 1: Copy raw data before any engineered columns
raw_df = flight.copy()

# STEP 2: First split → 85% train_val, 15% test
X_raw_trainval, X_raw_test = train_test_split(raw_df, test_size=0.15, random_state=42)

# STEP 3: Second split → train (70%) and validation (15%)
X_raw_train, X_raw_val = train_test_split(X_raw_trainval, test_size=0.1765, random_state=42)  # ≈ 15% of total

# STEP 4: Feature Engineering (apply same pipeline used in classifier)
X_train_fe = feature_engineering_regressor(X_raw_train)
X_val_fe   = feature_engineering_regressor(X_raw_val)
X_test_fe  = feature_engineering_regressor(X_raw_test)

# STEP 5: Extract regression target
y_train_reg = X_train_fe['arr_delay']
y_val_reg   = X_val_fe['arr_delay']
y_test_reg  = X_test_fe['arr_delay']

# STEP 6: Drop target from feature sets
X_train_fe.drop(['arr_delay'], axis=1, inplace=True)
X_val_fe.drop(['arr_delay'], axis=1, inplace=True)
X_test_fe.drop(['arr_delay'], axis=1, inplace=True)


C:\Users\omar yasser\AppData\Local\Temp\ipykernel_3916\109016724.py:40: RuntimeWarning: invalid value encountered in scalar divide
  lambda x: x['arr_del15'].sum() / x['arr_flights'].sum()
C:\Users\omar yasser\AppData\Local\Temp\ipykernel_3916\109016724.py:40: RuntimeWarning: invalid value encountered in scalar divide
  lambda x: x['arr_del15'].sum() / x['arr_flights'].sum()
C:\Users\omar yasser\AppData\Local\Temp\ipykernel_3916\109016724.py:40: RuntimeWarning: invalid value encountered in scalar divide
  lambda x: x['arr_del15'].sum() / x['arr_flights'].sum()


### Pre processing

In [9]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

def preprocessing_regressor(df, fit=True, scaler=None, encoder=None):
    """
    Preprocessing for Regression Task:
    - Drops leakage-prone and irrelevant columns
    - Encodes categorical features using OneHotEncoder
    - Scales numerical features using StandardScaler
    - Returns: processed_df, scaler, encoder
    """

    # Columns that leak or are irrelevant to regression modeling
    drop_cols = [
        'carrier_name', 'airport_name',
        'arr_flights', 'arr_del15',
        'carrier_ct', 'weather_ct', 'nas_ct', 'security_ct', 'late_aircraft_ct',
        'arr_cancelled', 'arr_diverted',
        'arr_delay',  # ✅ this is the target
        'carrier_delay', 'weather_delay', 'nas_delay',
        'security_delay', 'late_aircraft_delay', 'total_delay',
        'delay_ratio', 'high_delay_flag',
        'delay_risk_level', 'year_month', 'season_airport_combo'
    ]
    df = df.drop(columns=[col for col in drop_cols if col in df.columns], errors='ignore')

    # Fix dtype for any accidental categorical numerics
    to_str_cols = ['carrier_total_flights', 'airport_delay_rate']
    for col in to_str_cols:
        if col in df.columns and pd.api.types.is_categorical_dtype(df[col]):
            df[col] = df[col].astype(str)

    # Identify feature types
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
    numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

    # One-hot encode categoricals
    if fit:
        encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
        encoded_cat = encoder.fit_transform(df[categorical_cols])
    else:
        encoded_cat = encoder.transform(df[categorical_cols])
    encoded_cat_df = pd.DataFrame(encoded_cat, columns=encoder.get_feature_names_out(categorical_cols), index=df.index)

    # Scale numericals
    if fit:
        scaler = StandardScaler()
        scaled_num = scaler.fit_transform(df[numeric_cols])
    else:
        scaled_num = scaler.transform(df[numeric_cols])
    scaled_num_df = pd.DataFrame(scaled_num, columns=numeric_cols, index=df.index)

    # Merge
    processed_df = pd.concat([scaled_num_df, encoded_cat_df], axis=1)

    # Safety fill for any unexpected NaNs
    processed_df = processed_df.fillna(0)

    return processed_df, scaler, encoder


### Dumbing the encoder and the scaler and applying preprocessing

In [10]:
X_train_processed, scaler, encoder = preprocessing_regressor(X_train_fe, fit=True)

import joblib

# Save fitted encoder, scaler, and the correct feature order
joblib.dump(scaler, "scaler.pkl")
joblib.dump(encoder, "encoder.pkl")
joblib.dump(X_train_processed.columns.tolist(), "feature_order.pkl")


X_val_processed, _, _ = preprocessing_regressor(X_val_fe, fit=False, scaler=scaler, encoder=encoder)
X_test_processed, _, _ = preprocessing_regressor(X_test_fe, fit=False, scaler=scaler, encoder=encoder)


C:\Users\omar yasser\AppData\Local\Temp\ipykernel_3916\633148634.py:29: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if col in df.columns and pd.api.types.is_categorical_dtype(df[col]):
C:\Users\omar yasser\AppData\Local\Temp\ipykernel_3916\633148634.py:29: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if col in df.columns and pd.api.types.is_categorical_dtype(df[col]):
C:\Users\omar yasser\AppData\Local\Temp\ipykernel_3916\633148634.py:29: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if col in df.columns and pd.api.types.is_categorical_dtype(df[col]):


### Evaluation function and ploting

In [11]:
from sklearn.metrics import (
    mean_absolute_error, mean_squared_error, r2_score,
    median_absolute_error, mean_squared_log_error
)
import numpy as np

def evaluate_regression_metrics(y_true, y_pred, label="Set"):
    """Prints core regression metrics for a given dataset."""
    mae = mean_absolute_error(y_true, y_pred)
    medae = median_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)

    print(f"\n📊 {label} Set Metrics:")
    print(f"   - MAE:     {mae:.2f}")
    print(f"   - MedAE:   {medae:.2f}")
    print(f"   - RMSE:    {rmse:.2f}")
    print(f"   - R²:      {r2:.4f}")

    try:
        msle = mean_squared_log_error(np.clip(y_true, a_min=0, a_max=None),
                                      np.clip(y_pred, a_min=0, a_max=None))
        print(f"   - MSLE:    {msle:.4f}")
    except:
        print(f"   - MSLE:    Skipped (negative values detected)")
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

def plot_regression_diagnostics(X_fe_df, y_true, y_pred, label="Set"):
    """Draws correlation matrix and predicted-vs-actual scatter plot."""
    
    # Correlation matrix
    numeric_df = X_fe_df.select_dtypes(include=[np.number])
    corr = numeric_df.corr()
    plt.figure(figsize=(12, 10))
    sns.heatmap(corr, cmap='coolwarm', center=0, annot=False)
    plt.title(f"{label} Set | Correlation Matrix")
    plt.tight_layout()
    plt.show()

    # Predicted vs actual
    plt.figure(figsize=(6, 6))
    sns.scatterplot(x=y_true, y=y_pred, alpha=0.3)
    plt.xlabel("True Arrival Delay (min)")
    plt.ylabel("Predicted Arrival Delay (min)")
    plt.title(f"{label} Set | Predicted vs Actual")
    plt.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], '--', color='red')
    plt.grid(True)
    plt.tight_layout()
    plt.show()


### optuna tuning for hyper parameters

In [12]:
import optuna
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 200),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 1.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 1.0),
        "random_state": 42,
        "n_jobs": -1
    }

    model = XGBRegressor(**params)

    model.fit(
        X_train_processed, y_train_reg,
        eval_set=[(X_val_processed, y_val_reg)],
        verbose=False
    )

    preds = model.predict(X_val_processed)
    mae = mean_absolute_error(y_val_reg, preds)
    return mae

# Run the Optuna study
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=1)

# Best params → use for final model
best_params = study.best_params
best_params["random_state"] = 42
best_params["n_jobs"] = -1

print("✅ Best Params:", best_params)


d:\python\mid_project_raw\flight_delay_dashboard_project\envs\dashboard_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-06-11 20:24:30,761] A new study created in memory with name: no-name-bc92026c-c6d2-43eb-ae0c-12f39b39a341
[I 2025-06-11 20:24:40,295] Trial 0 finished with value: 1785.5391191928645 and parameters: {'n_estimators': 111, 'max_depth': 6, 'learning_rate': 0.09855988569133031, 'subsample': 0.7000692971979823, 'colsample_bytree': 0.9914280076289943, 'reg_alpha': 0.8971207068496274, 'reg_lambda': 0.9206149086096961}. Best is trial 0 with value: 1785.5391191928645.


✅ Best Params: {'n_estimators': 111, 'max_depth': 6, 'learning_rate': 0.09855988569133031, 'subsample': 0.7000692971979823, 'colsample_bytree': 0.9914280076289943, 'reg_alpha': 0.8971207068496274, 'reg_lambda': 0.9206149086096961, 'random_state': 42, 'n_jobs': -1}


### Training the model

In [13]:
from xgboost import XGBRegressor

# Step 1: Initialize the model (baseline config)
xgb_model = XGBRegressor(**best_params)


# Step 2: Train on training set
xgb_model.fit(X_train_processed, y_train_reg)

# Step 3: Predict on all sets
y_train_pred_xgb = xgb_model.predict(X_train_processed)
y_val_pred_xgb   = xgb_model.predict(X_val_processed)
y_test_pred_xgb  = xgb_model.predict(X_test_processed)

# Step 4: Evaluate with metrics (no plots)
evaluate_regression_metrics(y_train_reg, y_train_pred_xgb, label="Train")
evaluate_regression_metrics(y_val_reg, y_val_pred_xgb, label="Validation")
evaluate_regression_metrics(y_test_reg, y_test_pred_xgb, label="Test")



📊 Train Set Metrics:
   - MAE:     1610.01
   - MedAE:   591.08
   - RMSE:    3711.72
   - R²:      0.9140
   - MSLE:    2.2770

📊 Validation Set Metrics:
   - MAE:     1785.54
   - MedAE:   593.05
   - RMSE:    4509.92
   - R²:      0.8632
   - MSLE:    2.2240

📊 Test Set Metrics:
   - MAE:     1768.96
   - MedAE:   611.50
   - RMSE:    4515.27
   - R²:      0.8775
   - MSLE:    2.3754


### Tied ensemble but got me worse results in a more training time

In [15]:
'''from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

# Base learners including the final Optuna-tuned XGBoost
base_learners = [
    ("xgb", XGBRegressor(**best_params)),
    ("rf", RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)),
    ("lr", LinearRegression())
]

# Meta-learner (can be LinearRegression or Ridge)
meta_learner = LinearRegression()

# Build stacking ensemble
stacked_model = StackingRegressor(
    estimators=base_learners,
    final_estimator=meta_learner,
    passthrough=True,  # allows meta-learner to access original features
    n_jobs=-1
)

# Train ensemble
stacked_model.fit(X_train_processed, y_train_reg)

# Predict
y_train_pred_ens = stacked_model.predict(X_train_processed)
y_val_pred_ens   = stacked_model.predict(X_val_processed)
y_test_pred_ens  = stacked_model.predict(X_test_processed)

# Evaluate
evaluate_regression_metrics(y_train_reg, y_train_pred_ens, label="Train Ensemble")
evaluate_regression_metrics(y_val_reg, y_val_pred_ens, label="Validation Ensemble")
evaluate_regression_metrics(y_test_reg, y_test_pred_ens, label="Test Ensemble")
'''

'from sklearn.ensemble import StackingRegressor\nfrom sklearn.linear_model import LinearRegression\nfrom sklearn.ensemble import RandomForestRegressor\nfrom xgboost import XGBRegressor\n\n# Base learners including the final Optuna-tuned XGBoost\nbase_learners = [\n    ("xgb", XGBRegressor(**best_params)),\n    ("rf", RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)),\n    ("lr", LinearRegression())\n]\n\n# Meta-learner (can be LinearRegression or Ridge)\nmeta_learner = LinearRegression()\n\n# Build stacking ensemble\nstacked_model = StackingRegressor(\n    estimators=base_learners,\n    final_estimator=meta_learner,\n    passthrough=True,  # allows meta-learner to access original features\n    n_jobs=-1\n)\n\n# Train ensemble\nstacked_model.fit(X_train_processed, y_train_reg)\n\n# Predict\ny_train_pred_ens = stacked_model.predict(X_train_processed)\ny_val_pred_ens   = stacked_model.predict(X_val_processed)\ny_test_pred_ens  = stacked_model.predict(X_test_processed)\n

### Dumping the model

In [16]:
import joblib

# Save the model
joblib.dump(xgb_model, "model_regressor.pkl")

# Save the preprocessing pipeline
joblib.dump(scaler, "scaler_regressor.pkl")
joblib.dump(encoder, "encoder_regressor.pkl")

print("✅ XGBoost model, scaler, and encoder saved.")


✅ XGBoost model, scaler, and encoder saved.


### Comparison i made for the diffrent hyper parameters and models i used

In [27]:
import pandas as pd

metrics_table = pd.DataFrame({
    "Metric": ["MAE", "MedAE", "RMSE", "R² Score", "MSLE"],
    "XGBoost (Manual)": [2046.14, 552.17, 6223.33, 0.7673, 2.9971],
    "XGBoost (2 Trials)": [1876.63, 454.76, 5821.84, 0.7964, 0.7080],
    "XGBoost (10 Trials)": [1801.91, 417.69, 5830.47, 0.7958, 1.5963],
    "XGBoost (50 Trials)": [1738.37, 397.65, 5497.54, 0.8184, 1.5618],
    "Stacked Ensemble": [1768.62, 480.42, 5365.67, 0.8270, 3.7242]
})

metrics_table.to_csv("regression_metrics_summary.csv", index=False)
print("✅ Regression results saved to 'regression_metrics_summary.csv'")


✅ Regression results saved to 'regression_metrics_summary.csv'
